# Using TempestExtremes' NodeFileCompose to extract cyclone-centered snapshots

In this example, we extract SLP snapshots along the tracks in `TC_tracks/um_glm_n1280_GAL9_sample.csv`, which is a subset of three tracks.

Snapshots should be retrieved for all TC points listed in the tracks csv, over a radius of 10°GCD around the cyclone center. 
Snapshots must be obtained from data at a resolution close to the simulation resolution in the tropics, 
and choosing NodeFileCompose parameters so that the resolution of the snapshots is similar to that of the simulation.

In this case, we have pre-processed the data over a 0.1x0.1° grid. 

In [ ]:
# Activate your conda environment which must include tempestextremes + the hackathon packages such as intake, easygems etc.
conda activate hackathon
# If you have a message saying you need to initalize conda: 
## create a new cell with the `conda init` command
## Run that cell
## Delete the cell
## Restart your kernel
## Run the notebook again

In [ ]:
# Script Parameters
run='um_glm_n1280_GAL9' # Code of your simulation (must be the reference in the catalog)
var="psl"

# Folders
scr_dir='/work/scratch-nopw2/sbourdin/' # Change to your own scratch/temporary folder
data_pp_dir=$scr_dir/$run/data_pp # dir to store preprocessed (pp) files

# Input
#tracks_file="../TC_tracks/$run.csv"
tracks_file="../TC_tracks/${run}_sample.csv"

# Output
if ! [ -d ../snapshots/${var}/ ]; then mkdir ../snapshots/${var}/; fi
snaps_file="../snapshots/${var}/${run}_${var}.nc"

Before you start, make sure you have preprocessed the data you need with `TE_pre-processing.ipynb`: You must have the fields covering all the time steps in your tracks.

In [ ]:
# List pre-processed files for this variable
ls $data_pp_dir/*$var*

In [ ]:
## Prepare file list
flist=`ls $data_pp_dir/*$var*`
INPUT=""
for f in $flist
do
INPUT="$INPUT$f;"
done
INPUT=${INPUT:0:-1} # Remove last ;

In NodeFileCompose, we'll use an XY (square) box centered on the cyclone center. 
To avoid loss of information as much as possible, `dx` should be set to be close to the model resolution. Note `dx` is in °GCD. 
Then, adjust `resx` to make sure 1000km=10°GCD.
For example, for a 10km grid, `dx=0.1` and `resx=100`.

In [ ]:
NodeFileCompose \
--in_nodefile "$tracks_file" \
--in_nodefile_type "SN" \
--in_fmt "(auto)" \
--in_data "$INPUT" \
--out_data "$snaps_file" \
--var "psl" \
--varout "psl" \
--out_grid "XY" \
--dx 0.1 \
--resx 100 \
--snapshots